# Cardiology LightRAG Clinical QA System

This notebook demonstrates the CARDIO-LR system for answering cardiology-related questions with patient context integration.

## Setup and Dependency Check

First, let's ensure all required packages are installed correctly.

In [1]:
import sys
import os
import subprocess
import pkg_resources

# Define required packages
required_packages = [
    'torch',
    'torch-geometric',
    'torch_sparse',
    'transformers', 
    'sentence-transformers',
    'faiss-cpu',
    'pandas',
    'numpy',
    'networkx',
    'scikit-learn',
    'rouge-score',
    'tqdm',
    'ipywidgets',
    'ipykernel',
    'matplotlib'
]

def install_package(package):
    print(f"Installing {package}...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

# Check for missing packages and install them
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = []

for package in required_packages:
    package_name = package.split('>=')[0].strip() if '>=' in package else package
    if package_name.replace('-', '_') not in installed:
        missing.append(package)

if missing:
    print(f"Missing packages: {', '.join(missing)}")
    for package in missing:
        try:
            install_package(package)
        except Exception as e:
            print(f"Failed to install {package}: {str(e)}")
            print(f"Try installing {package} manually with: pip install {package}")
    print("\nNote: You may need to restart the kernel for changes to take effect.")
else:
    print("All required packages already installed!")

# Special check for faiss-cpu since it's critical
try:
    import faiss
    print(f"faiss-cpu successfully imported (version: {faiss.__version__})")
except ImportError:
    print("faiss-cpu not found, installing...")
    try:
        install_package('faiss-cpu')
        print("Please restart the kernel after installation completes")
    except Exception as e:
        print(f"Failed to install faiss-cpu: {str(e)}")
        print("Try installing manually with: pip install faiss-cpu")

Missing packages: torch-geometric, torch_sparse, sentence-transformers, faiss-cpu, scikit-learn, rouge-score
Installing torch-geometric...
Defaulting to user installation because normal site-packages is not writeable
Installing torch_sparse...
Defaulting to user installation because normal site-packages is not writeable
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for torch_sparse: started


  DEPRECATION: torch_sparse is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


  Running setup.py install for torch_sparse: finished with status 'error'
Failed to install torch_sparse: Command '['/usr/bin/python3.11', '-m', 'pip', 'install', 'torch_sparse']' returned non-zero exit status 1.
Try installing torch_sparse manually with: pip install torch_sparse
Installing sentence-transformers...


  error: subprocess-exited-with-error
  
  × Running setup.py install for torch_sparse did not run successfully.
  │ exit code: 1
  ╰─> [66 lines of output]
      running install
      /usr/lib/python3.11/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-311
      creating build/lib.linux-x86_64-cpython-311/torch_sparse
      copying torch_sparse/__init__.py -> build/lib.linux-x86_64-cpython-311/torch_sparse
      copying torch_sparse/add.py -> build/lib.linux-x86_64-cpython-311/torch_sparse
      copying torch_sparse/bandwidth.py -> build/lib.linux-x86_64-cpython-311/torch_sparse
      copying torch_sparse/cat.py -> build/lib.linux-x86_64-cpython-311/torch_sparse
      copying torch_sparse/coalesce.py -> build/lib.linux-x86_64-cpython-311/torch_

Defaulting to user installation because normal site-packages is not writeable
Installing faiss-cpu...
Defaulting to user installation because normal site-packages is not writeable
Installing scikit-learn...
Defaulting to user installation because normal site-packages is not writeable
Installing rouge-score...
Defaulting to user installation because normal site-packages is not writeable

Note: You may need to restart the kernel for changes to take effect.
faiss-cpu successfully imported (version: 1.11.0)


In [2]:
# Check if dependencies are available before continuing
try:
    import torch
    import numpy as np
    import pandas as pd
    import networkx as nx
    import ipywidgets as widgets
    from IPython.display import display, Markdown
    print("✅ Core dependencies loaded successfully!")
except ImportError as e:
    print(f"❌ Error importing dependencies: {str(e)}")
    print("Please install missing packages and restart the notebook kernel.")

✅ Core dependencies loaded successfully!


In [3]:
# Add the parent directory to path so we can import modules from there
import sys
import os
if os.path.abspath('..') not in sys.path:
    sys.path.append(os.path.abspath('..'))

# Try importing the CardiologyLightRAG class with error handling
try:
    from pipeline import CardiologyLightRAG
    print("✅ Successfully imported CardiologyLightRAG!")
except ImportError as e:
    print(f"❌ Error importing CardiologyLightRAG: {str(e)}")
    print("This might be due to missing dependencies or issues with the pipeline module.")
    import traceback
    traceback.print_exc()

import time

✅ Successfully imported CardiologyLightRAG!


In [4]:
# Initialize system with error handling
try:
    print("Initializing CardiologyLightRAG system...")
    # Remove the device argument, as HybridRetriever does not accept it
    system = CardiologyLightRAG()
    print("✅ System initialized successfully!")
except Exception as e:
    print(f"❌ Error initializing CardiologyLightRAG: {str(e)}")
    print("\nTroubleshooting tips:")
    print("1. Check if all dependencies are installed")
    print("2. Verify that model files exist in the expected directories")
    print("3. Look for any error messages above that might indicate specific issues")
    import traceback
    traceback.print_exc()
    # Create a dummy system for UI to work
    class DummySystem:
        def process_query(self, query, patient_context):
            return ("Error: System could not be initialized. Please fix the issues above.",
                    "No explanation available.")
    system = DummySystem()

Initializing CardiologyLightRAG system...
Initializing Cardiology LightRAG system...
Using device: cuda
Loading retrieval system...


No sentence-transformers model found with name dmis-lab/biobert-v1.1. Creating a new one with mean pooling.


Loading knowledge graph components...
Subgraph extractor using device: cuda
Loading integrated knowledge graph...
Knowledge graph loaded with 3454254 nodes and 6648456 edges
Creating node and relation mappings...
Created mappings for 3454254 nodes and 10 relation types
Loading GNN model for path selection...
No pre-trained GNN model found. Using untrained model.
Loading generator and validation components...
Loading AutoTokenizer and AutoModelForCausalLM...
Generation parameters set: attention_mask=True, pad_token_id=50256


Device set to use cpu
Device set to use cpu


TraceabilityLogger: Successfully loaded original knowledge graph
Subgraph extractor using device: cuda
Loading integrated knowledge graph...
Knowledge graph loaded with 3454254 nodes and 6648456 edges
Creating node and relation mappings...
Created mappings for 3454254 nodes and 10 relation types
Cardiology LightRAG system initialized and ready!
✅ System initialized successfully!


In [5]:
# Create UI components
question_input = widgets.Textarea(
    value='What are the first-line treatments for stable angina?',
    placeholder='Enter a cardiology-related question...',
    description='Question:',
    layout={'width': '90%', 'height': '100px'}
)

In [6]:
patient_context = widgets.Textarea(
    value='Patient has diabetes and hypertension',
    placeholder='Enter patient context (conditions, allergies, meds)...',
    description='Patient:',
    layout={'width': '90%', 'height': '80px'}
)

In [7]:
submit_btn = widgets.Button(description="Get Clinical Answer", button_style='success')
output_area = widgets.Output()
explanation_area = widgets.Output()

In [8]:
def on_submit_clicked(b):
    with output_area:
        output_area.clear_output()
        explanation_area.clear_output()
        
        start_time = time.time()
        print("Processing your cardiology query...")
        
        try:
            # Process query - convert patient_context text to a structured format
            patient_text = patient_context.value.strip()
            # Convert simple text to the expected dictionary format
            patient_data = {
                "description": patient_text,
                "conditions": [],
                "medications": [],
                "allergies": []
            }
            
            # Extract conditions, medications, allergies if explicitly mentioned
            if "diabetes" in patient_text.lower():
                patient_data["conditions"].append("diabetes")
            if "hypertension" in patient_text.lower():
                patient_data["conditions"].append("hypertension")
                
            answer, explanation = system.process_query(
                question_input.value, 
                patient_data
            )
            
            end_time = time.time()
            duration = end_time - start_time
            
            # Display results
            display(Markdown(f"### Clinical Answer"))
            display(Markdown(f"{answer}"))
            display(Markdown(f"*Generated in {duration:.2f} seconds*"))
            
            # Show explanation
            with explanation_area:
                display(Markdown(f"### Clinical Reasoning Report"))
                display(Markdown(explanation))
                
        except Exception as e:
            display(Markdown(f"### Error"))
            display(Markdown(f"An error occurred: {str(e)}"))
            display(Markdown("Please check that all dependencies are installed correctly."))
            import traceback
            traceback.print_exc()

In [9]:
submit_btn.on_click(on_submit_clicked)

In [10]:
# Display UI
display(widgets.VBox([
    widgets.HTML("<h1>Cardiology LightRAG Clinical QA System</h1>"),
    widgets.HTML("<p>Ask cardiology-related questions with patient-specific context</p>"),
    question_input,
    patient_context,
    submit_btn,
    output_area,
    explanation_area
]))

In [11]:
# Verify the expected arguments for RGCNReasoner.__init__
from inspect import signature
try:
    from gnn.rgcn_model import RGCNReasoner
    print('RGCNReasoner imported successfully!')
    print('Expected arguments for RGCNReasoner.__init__():')
    print(signature(RGCNReasoner.__init__))
except ImportError as e:
    print(f'Error importing RGCNReasoner: {str(e)}')
    print('Ensure the gnn module is correctly installed and accessible.')
except Exception as e:
    print(f'Unexpected error: {str(e)}')

RGCNReasoner imported successfully!
Expected arguments for RGCNReasoner.__init__():
(self, num_nodes, num_relations, embedding_dim=128, hidden_dim=256, num_bases=None, dropout=0.2, **kwargs)
